In [21]:
import os
import random
import numpy as np
import tensorflow as tf
from astropy.io import fits
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import confusion_matrix, classification_report

# Create TensorFlow datasets using the loaded data
def create_tf_dataset(spectra_data, labels, batch_size=64):
    dataset = tf.data.Dataset.from_tensor_slices((spectra_data, labels))
    dataset = dataset.shuffle(buffer_size=len(spectra_data)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def removenan(train_spectra, train_labels, val_spectra, val_labels): 
    # Convert lists to NumPy arrays and remove first 10 spectra to avoid NaNs
    train_spectraa = np.array(train_spectra[10:, :])
    train_labelsa = np.array(train_labels[10:])
    val_spectraa = np.array(val_spectra[10:, :])
    val_labelsa = np.array(val_labels[10:])

    # Remove nan values from spectra
    mask_train = ~np.isnan(train_spectraa).any(axis=1)
    train_spectranan = train_spectraa[mask_train]
    train_labelsnan = train_labelsa[mask_train]

    mask_val = ~np.isnan(val_spectraa).any(axis=1)
    val_spectranan = val_spectraa[mask_val]
    val_labelsnan = val_labelsa[mask_val]

    # Create TensorFlow datasets
    train_dataset = create_tf_dataset(train_spectranan, train_labelsnan)
    val_dataset = create_tf_dataset(val_spectranan, val_labelsnan)

    return train_dataset, val_dataset, val_spectranan, val_labelsnan


def normalize_spectra(spectra):
    """Normalize spectra by dividing by the mean and applying the natural logarithm."""
    mean_value = np.mean(spectra)
    std_value = np.std(spectra)
    if std_value == 0:
        return spectra  # Avoid division by zero
    return (spectra - mean_value) / (std_value + 1e-7)

def load_single_spectrum(file_path, target_length=3748):
    """Load and normalize a single spectrum from a FITS file, truncating or padding to target_length."""
    try:
        with fits.open(file_path) as hdul:
            spectra = hdul[0].data[0]  # Assumes the spectra are in the first HDU
            spectra = normalize_spectra(spectra)

            # Truncate or pad spectra to ensure uniform length
            if len(spectra) > target_length:
                spectra = spectra[:target_length]  # Truncate
            else:
                spectra = np.pad(spectra, (0, max(0, target_length - len(spectra))), mode='constant')  # Pad with zeros

            return spectra
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None  # Return None if there's an error

def tf_load_single_spectrum(file_path, target_length=3748):
    """TensorFlow wrapper for loading a single spectrum using py_function."""
    spectra = tf.py_function(load_single_spectrum, [file_path, target_length], tf.float32)
    spectra.set_shape([target_length])  # Set the shape explicitly for TensorFlow to optimize
    return spectra

def create_tf_dataset_from_files(file_list, labels, batch_size=64, target_length=3748):
    """Create TensorFlow dataset with parallelized loading and optimization."""
    dataset = tf.data.Dataset.from_tensor_slices((file_list, labels))

    def load_and_preprocess(file_path, label):
        spectra = tf_load_single_spectrum(file_path, target_length)
        return spectra, label

    # Apply parallelism and optimizations
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()  # Cache the dataset after loading it once
    dataset = dataset.shuffle(buffer_size=len(file_list))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)  # Prefetch to overlap data loading and training

    return dataset

def generate_datasets_from_preseparated(limit_per_dir=50000, batch_size=64, target_length=3748):
    """Generates training and validation datasets directly from pre-separated directories."""
    train_base_dir = os.path.join(os.getcwd(), "training_set")
    val_base_dir = os.path.join(os.getcwd(), "validation_set")

    # Load files and labels from the respective directories
    train_files, train_labels = generate_file_list_from_directories([train_base_dir], limit_per_dir)
    val_files, val_labels = generate_file_list_from_directories([val_base_dir], limit_per_dir)

    # Create TensorFlow datasets with parallel loading
    train_dataset = create_tf_dataset_from_files(train_files, train_labels, batch_size, target_length)
    val_dataset = create_tf_dataset_from_files(val_files, val_labels, batch_size, target_length)

    return train_dataset, train_labels, val_dataset, val_labels

def create_convnet(input_shape, num_classes, 
                   num_filters=[128, 128, 128, 128, 128, 128, 128, 128], 
                   kernel_size=(9,),
                   dense_units1=256, 
                   dense_units2=128,
                   dense_units3=64,
                   dropout_rate=0.2,
                   padding='same'):
    model = tf.keras.models.Sequential()
    
    # First convolutional layer
    model.add(tf.keras.layers.Conv1D(filters=num_filters[0], kernel_size=kernel_size, 
                                     activation='relu', input_shape=input_shape, padding=padding))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    
    # Additional convolutional layers
    for filters in num_filters[1:]:
        model.add(tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, 
                                         activation='relu', padding=padding))
        model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
        model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    
    # Flatten the output and add dense layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units1, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))

    # Adding additional dense layers
    if dense_units2:
        model.add(tf.keras.layers.Dense(units=dense_units2, activation='relu'))
        model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    if dense_units3:
        model.add(tf.keras.layers.Dense(units=dense_units3, activation='relu'))
        model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    
    # Output layer
    model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

    # Compile the model
    optimizer_ = tf.keras.optimizers.AdamW(learning_rate=1e-4) 
    model.compile(optimizer=optimizer_, 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

def train_convnet(model, train_dataset, val_dataset, epochs=1, batch_size=32, patience=5):
    """Train the model with early stopping."""
    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    
    # Fit the model
    history = model.fit(train_dataset,
                        validation_data=val_dataset,
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping])
    
    return history

def load_single_spectrum(file_path, target_length=3748):
    """Load and normalize a single spectrum from a FITS file, truncating or padding to target_length."""
    try:
        # Ensure file_path is decoded from bytes to string
        file_path = file_path.numpy().decode('utf-8') if isinstance(file_path, tf.Tensor) else file_path
        
        with fits.open(file_path) as hdul:
            spectra = hdul[0].data[0]  # Assumes the spectra are in the first HDU
            spectra = normalize_spectra(spectra)

            # Truncate or pad spectra to ensure uniform length
            if len(spectra) > target_length:
                spectra = spectra[:target_length]  # Truncate
            else:
                spectra = spectra

            return spectra
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        # Return a zero-filled array of the expected shape in case of error
        return np.zeros(target_length, dtype=np.float32)

def tf_load_single_spectrum(file_path, target_length=3748):
    """TensorFlow wrapper for loading a single spectrum using py_function."""
    spectra = tf.py_function(load_single_spectrum, [file_path, target_length], tf.float32)
    spectra.set_shape([target_length])  # Set the shape explicitly for TensorFlow to optimize
    return spectra

def generate_file_list_from_directories(base_dirs, limit_per_dir=10000):
    """Generates a list of files and labels from the pre-separated directories."""
    spectra_dirs = {
        "gal_spectra": 0,  # Label 0 for galaxies
        "star_spectra": 1,  # Label 1 for stars
        "agn_spectra": 2,   # Label 2 for AGNs
        "bin_spectra": 3    # Label 3 for binary stars
    }

    file_list = []
    labels = []

    print("Gathering FITS files from pre-separated directories...")
    for dir_name, label in spectra_dirs.items():
        for base_dir in base_dirs:
            dir_path = os.path.join(base_dir, dir_name)
            dir_files = [os.path.join(root, file) 
                         for root, _, files in os.walk(dir_path) 
                         for file in files ]

            # Randomly select files up to the limit
            selected_files = random.sample(dir_files, min(len(dir_files), limit_per_dir))

            # Append selected files and their labels
            file_list.extend(selected_files)
            labels.extend([label] * len(selected_files))

    print(f"Total spectra files collected: {len(file_list)}")
    return file_list, labels


In [5]:
def print_confusion_matrix(convnet_model, val_spectranan, val_labelsnan):   
    # Make predictions on the validation/test dataset
    val_predictions = convnet_model.predict(val_spectranan)

    # Convert the predictions to class labels (assuming one-hot encoding)
    predicted_labels = np.argmax(val_predictions, axis=1)

    # Convert true labels if they are in one-hot encoded format
    true_labels = np.array(val_labelsnan)  # Assuming val_labels is already numeric


    # Generate the confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    # Print the confusion matrix
    print("Confusion Matrix:")
    print(conf_matrix)

    # Optionally, print a classification report for more metrics
    print("\nClassification Report:")
    print(classification_report(true_labels, predicted_labels))

    # Plot the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Galaxy', 'Star', 'AGN', 'Binary Star'], yticklabels=['Galaxy', 'Star', 'AGN', 'Binary Star'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# experiment

In [18]:
def generate_datasets_from_preseparated(limit_per_dir=10000, batch_size=64, target_length=3748):
    """Generates training and validation datasets directly from pre-separated directories."""
    train_base_dir = os.path.join(os.getcwd(), "training_set")
    val_base_dir = os.path.join(os.getcwd(), "validation_set")

    # Load files and labels from the respective directories
    train_files, train_labels = generate_file_list_from_directories([train_base_dir], limit_per_dir)
    val_files, val_labels = generate_file_list_from_directories([val_base_dir], limit_per_dir)

    # Create TensorFlow datasets with parallel loading
    train_dataset, val_dataset, train_spectra, train_labels_clean = removenan(
        train_files, train_labels, val_files, val_labels
    )

    return train_dataset, val_dataset, train_files, train_labels

def removenan(train_spectra, train_labels, val_spectra, val_labels): 
    # Convert lists to NumPy arrays and remove first 10 spectra to avoid NaNs
    train_spectraa = np.array(train_spectra[10:, :])
    train_labelsa = np.array(train_labels[10:])
    val_spectraa = np.array(val_spectra[10:, :])
    val_labelsa = np.array(val_labels[10:])

    # Remove nan values from spectra
    mask_train = ~np.isnan(train_spectraa).any(axis=1)
    train_spectranan = train_spectraa[mask_train]
    train_labelsnan = train_labelsa[mask_train]
    
    print(f"Removed NaNs from training data: {len(train_spectraa) - len(train_spectranan)} entries")

    mask_val = ~np.isnan(val_spectraa).any(axis=1)
    val_spectranan = val_spectraa[mask_val]
    val_labelsnan = val_labelsa[mask_val]

    print(f"Removed NaNs from validation data: {len(val_spectraa) - len(val_spectranan)} entries")

    # Create TensorFlow datasets
    train_dataset = create_tf_dataset(train_spectranan, train_labelsnan)
    val_dataset = create_tf_dataset(val_spectranan, val_labelsnan)

    return train_dataset, val_dataset, train_spectranan, val_labelsnan



In [20]:
def remove_nan_from_datasets(train_dataset, val_dataset):
    """Remove NaN values from TensorFlow datasets."""
    
    def filter_nan(spectra, label):
        """Filter out NaN spectra."""
        mask = tf.reduce_all(tf.math.is_finite(spectra), axis=-1)
        return tf.boolean_mask(spectra, mask), tf.boolean_mask(label, mask)

    # Apply the filtering function to both train and validation datasets
    train_dataset = train_dataset.map(filter_nan)
    val_dataset = val_dataset.map(filter_nan)

    return train_dataset, val_dataset


In [22]:
# Example usage
num_labels = 4
filters_20 = [512, 512, 512, 512, 512, 512, 256, 256, 256, 128]
input_shape = (3748, 1)

# Create the model
model_20 = create_convnet(input_shape=input_shape, num_classes=num_labels, num_filters=filters_20)

# Generate datasets
train_dataset, val_dataset = generate_datasets_from_preseparated()

# Remove NaN values from datasets
train_dataset, val_dataset = remove_nan_from_datasets(train_dataset, val_dataset)

# Train the model
#train_convnet(model_20, train_dataset, val_dataset, epochs=50, patience=5)


/home/jcwind/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Gathering FITS files from pre-separated directories...
Total spectra files collected: 31699
Gathering FITS files from pre-separated directories...
Total spectra files collected: 1600


ValueError: too many values to unpack (expected 2)

In [19]:
# Example usage
train_dataset, train_labels, val_dataset, val_labels  = generate_datasets_from_preseparated(limit_per_dir=2000)

Gathering FITS files from pre-separated directories...
Total spectra files collected: 7699
Gathering FITS files from pre-separated directories...
Total spectra files collected: 1600


TypeError: list indices must be integers or slices, not tuple

In [17]:
# After loading your spectra and labels
train_dataset, val_dataset, val_spectranan, val_labelsnan  = removenan(train_dataset, train_labels, val_dataset, val_labels)


TypeError: '_PrefetchDataset' object is not subscriptable

In [4]:
# Example usage
num_labels = 4
filters_20 = [512, 512, 512, 512, 512, 512, 256, 256, 256, 128]
input_shape = (3748, 1)

# Create the model
model_20 = create_convnet(input_shape=input_shape, num_classes=num_labels, num_filters=filters_20)

# Generate datasets
train_dataset, val_dataset = generate_datasets_from_preseparated()

# Train the model
train_convnet(model_20, train_dataset, val_dataset, epochs=50, patience=5)

/home/jcwind/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1727875741.728383     932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727875741.885619     932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727875741.885693     932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727875741.889103     932 cuda_executor.

Gathering FITS files from pre-separated directories...
Total spectra files collected: 31699
Gathering FITS files from pre-separated directories...
Total spectra files collected: 1600
Epoch 1/50


2024-10-02 14:29:16.764053: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 2207 of 31699
2024-10-02 14:29:36.765683: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 6448 of 31699
2024-10-02 14:29:56.767746: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 10657 of 31699
2024-10-02 14:30:16.764133: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 14806 of 31699
2024-10-02 14:30:26.766728: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 16772 of 31699
2024-10-02 14:30:46.762481: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may 

495/496 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - accuracy: 0.5283 - loss: 1.0000

2024-10-02 14:34:29.583310: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[512,512,1,9]{3,2,1,0}, u8[0]{0}) custom-call(f32[512,19,1,1874]{3,2,1,0}, f32[512,19,1,1874]{3,2,1,0}), window={size=1x1874 pad=0_0x4_4}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false} is taking a while...
2024-10-02 14:34:31.972951: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 3.389752948s
Trying algorithm eng0{} for conv (f32[512,512,1,9]{3,2,1,0}, u8[0]{0}) custom-call(f32[512,19,1,1874]{3,2,1,0}, f32[512,19,1,1874]{3,2,1,0}), window={size=1x1874 pad=0_0x4_4}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_ope

496/496 ━━━━━━━━━━━━━━━━━━━━ 353s 322ms/step - accuracy: 0.5286 - loss: 0.9995 - val_accuracy: 0.5794 - val_loss: 0.9220
Epoch 2/50
496/496 ━━━━━━━━━━━━━━━━━━━━ 127s 255ms/step - accuracy: 0.6510 - loss: 0.7071 - val_accuracy: 0.7344 - val_loss: 0.6488
Epoch 3/50
496/496 ━━━━━━━━━━━━━━━━━━━━ 129s 260ms/step - accuracy: 0.6933 - loss: 0.5955 - val_accuracy: 0.7144 - val_loss: 0.6085
Epoch 4/50
496/496 ━━━━━━━━━━━━━━━━━━━━ 131s 263ms/step - accuracy: 0.7179 - loss: 0.5437 - val_accuracy: 0.7219 - val_loss: 0.5891
Epoch 5/50
496/496 ━━━━━━━━━━━━━━━━━━━━ 130s 262ms/step - accuracy: 0.7462 - loss: 0.5080 - val_accuracy: 0.7544 - val_loss: 0.5041
Epoch 6/50
496/496 ━━━━━━━━━━━━━━━━━━━━ 130s 263ms/step - accuracy: 0.7559 - loss: 0.4811 - val_accuracy: 0.7725 - val_loss: 0.4887
Epoch 7/50
496/496 ━━━━━━━━━━━━━━━━━━━━ 131s 263ms/step - accuracy: 0.7768 - loss: 0.4518 - val_accuracy: 0.7794 - val_loss: 0.4818
Epoch 8/50
496/496 ━━━━━━━━━━━━━━━━━━━━ 132s 267ms/step - accuracy: 0.7857 - loss: 0.44

In [15]:
print_confusion_matrix(model_20, val_files, val_labels)

ValueError: Unrecognized data type: x=['/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/565907232', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/342501214', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/730705007', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/496311210', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/602214124', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/500005125', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/403703143', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/526003208', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/180712009', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/629715138', '/home/jcwind/Star Classifier/Star-Classifier/validation_set/gal_spectra/397305085', '/home/jcwind/Star Classif